In [1]:
import rospkg
import yaml
from utils.grasp_utils import *
from utils.nav_utils import *
from utils.know_utils import *
import rospy

In [2]:
rospy.init_node('knowledge')

In [3]:
def read_yaml(known_locations_file='/receptionist_knowledge2.yaml'):
    rospack = rospkg.RosPack()
    file_path = rospack.get_path('config_files') + known_locations_file

    with open(file_path, 'r') as file:
        content = yaml.safe_load(file)
    return content

In [4]:
knowledge = read_yaml()

In [5]:
knowledge

{'People': {'Guest_1': {'drink': 'Juice', 'location': 'None', 'name': 'Tomas'},
  'Guest_2': {'drink': 'Coffee', 'location': 'None', 'name': 'Pedro'},
  'Guest_3': {'drink': 'Juice', 'location': 'Place_2', 'name': 'Jose'},
  'Guest_4': {'drink': 'Beer', 'location': 'None', 'name': 'Juan'},
  'Host': {'drink': 'Coke', 'location': 'Place_1', 'name': 'Ruben'}},
 'Places': {'Place_0': {'location': {'theta': 0.0, 'x': 0.0, 'y': 0.0},
   'occupied': 'occupied'},
  'Place_1': {'location': {'theta': 0.05, 'x': 9.71, 'y': -3.14},
   'occupied': 'Host'},
  'Place_2': {'location': {'theta': 4.72, 'x': 9.76, 'y': -3.31},
   'occupied': 'Guest_3'},
  'Place_3': {'location': {'theta': 0.06, 'x': 9.75, 'y': -2.47},
   'occupied': 'None'}}}

In [6]:
def write_yaml(new_knowledge):
    rospack = rospkg.RosPack()
    file_path = rospack.get_path('config_files') + '/receptionist_knowledge2.yaml'
    with open(file_path, 'w') as file:
        documents = yaml.dump(new_knowledge, file)

In [7]:
def update_occupancy(found = 'None', place = 'None'):
    #Use: hsr found someone on a supposed empty place 
    knowledge = read_yaml()
    # found is a name
    if found != ' None' and place != 'None':
        #get guest number of this someone
        guest = [key for key, person_dict in knowledge['People'].items() if person_dict.get('name') == found]
        #get the seat that guest used to have
        seat = [key for key, places_dict in knowledge['Places'].items() if places_dict.get('occupied') == guest[0]]
        #change knowledge
        knowledge['People'][guest[0]]['location'] = place
        knowledge['Places'][seat[0]]['occupied'] = 'None'
        knowledge['Places'][place]['occupied'] = guest[0]
        write_yaml(knowledge)
        return True
    else: return False
    

In [8]:
def find_empty_places():
    knowledge = read_yaml()
    seat = [key for key, places_dict in knowledge['Places'].items() if places_dict.get('occupied') == 'None']
    loc = list(knowledge['Places'][seat[0]]['location'].values())
    t,x,y = loc
    return seat[0], [x,y,t]

In [9]:
def assign_occupancy(who = 'None', where = 'None'):
    #Use: hsr found a empty place for the new guest
    knowledge = read_yaml()
    if who != ' None' and where != 'None':
        guest = [key for key, person_dict in knowledge['People'].items() if person_dict.get('name') == who]
        knowledge['People'][guest[0]]['location'] = where
        knowledge['Places'][where]['occupied'] = guest[0]
        write_yaml(knowledge)
        return True
    else:
        return False

In [10]:
knowledge

{'People': {'Guest_1': {'drink': 'Juice', 'location': 'None', 'name': 'Tomas'},
  'Guest_2': {'drink': 'Coffee', 'location': 'None', 'name': 'Pedro'},
  'Guest_3': {'drink': 'Juice', 'location': 'Place_2', 'name': 'Jose'},
  'Guest_4': {'drink': 'Beer', 'location': 'None', 'name': 'Juan'},
  'Host': {'drink': 'Coke', 'location': 'Place_1', 'name': 'Ruben'}},
 'Places': {'Place_0': {'location': {'theta': 0.0, 'x': 0.0, 'y': 0.0},
   'occupied': 'occupied'},
  'Place_1': {'location': {'theta': 0.05, 'x': 9.71, 'y': -3.14},
   'occupied': 'Host'},
  'Place_2': {'location': {'theta': 4.72, 'x': 9.76, 'y': -3.31},
   'occupied': 'Guest_3'},
  'Place_3': {'location': {'theta': 0.06, 'x': 9.75, 'y': -2.47},
   'occupied': 'None'}}}

In [11]:
update_occupancy('Juan','Place_6')


IndexError: list index out of range

In [68]:
assign_occupancy('Pedro', 'Place_5')

True

         at line 278 in /tmp/binarydeb/ros-noetic-tf2-0.7.6/src/buffer_core.cpp
         at line 278 in /tmp/binarydeb/ros-noetic-tf2-0.7.6/src/buffer_core.cpp
         at line 278 in /tmp/binarydeb/ros-noetic-tf2-0.7.6/src/buffer_core.cpp
         at line 278 in /tmp/binarydeb/ros-noetic-tf2-0.7.6/src/buffer_core.cpp
         at line 278 in /tmp/binarydeb/ros-noetic-tf2-0.7.6/src/buffer_core.cpp
         at line 278 in /tmp/binarydeb/ros-noetic-tf2-0.7.6/src/buffer_core.cpp


In [12]:
find_empty_places()


('Place_3', [9.75, -2.47, 0.06])

In [13]:
def add_guest(name, drink):
    knowledge = read_yaml()
    guests_len = len(knowledge['People'])
    new_guest = f'Guest_{guests_len}'
    knowledge['People'][new_guest] = {'drink': drink, 'location': 'None', 'name': name}
    write_yaml(knowledge)
    return True

In [25]:
add_guest('Paco', 'Soda')

True

In [14]:
def add_place():
    try:
        tf_manager = TF_MANAGER()
        rospy.sleep(0.8)
        trans, rot = tf_manager.getTF( target_frame='base_link')
        _, _, theta = tf.transformations.euler_from_quaternion(rot)
        places_len = len(knowledge['Places'])
        new_place = f'Place_{places_len}'
        knowledge['Places'][new_place] = {'location': {'theta': round(theta%(6.283),2), 'x': round(trans[0],2), 'y': round(trans[1],2)}, 'occupied':'None'}
        write_yaml(knowledge)
        return True
    except:
        return False

In [39]:
add_place()

True

In [15]:
base = OMNIBASE()

In [16]:

place, loc= find_empty_places()

In [17]:
place

'Place_3'

In [18]:
base.move_base(*loc, time_out=30)

header: 
  seq: 0
  stamp: 
    secs: 0
    nsecs:         0
  frame_id: ''
goal_id: 
  stamp: 
    secs: 0
    nsecs:         0
  id: ''
goal: 
  x: 9.75
  y: -2.47
  yaw: 0.06
  timeout: 30
  known_location: "None"


3

In [40]:
update_occupancy('Ruben', 'Place_2')

IndexError: list index out of range

In [50]:
assign_occupancy('Jose', 'Place_2')

True

In [19]:
import face_recognition


In [ ]:
import numpy as np

In [1]:
from utils.nav_utils import *
import rospy

In [2]:
rospy.init_node('new_nav')

In [3]:
base = OMNIBASE()
tf_man = TF_MANAGER()

In [4]:
goal,_ = tf_man.getTF(target_frame='base_link')

In [5]:
base.navigate(goal[0], -2.0,0.0)

In [3]:
get_waiting_guests()

(0, '')